In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tables as tb
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from tqdm.notebook import tqdm, trange

In [2]:
from lmfit import Model, create_params
from lmfit.models import GaussianModel

mod = GaussianModel()


In [3]:
from scipy.special import erf


def damp_osci(x, x0, oa, rg, ro, phase, sigma, w):
    damp = oa * np.exp(-(x-x0)/rg)*np.cos((x-x0)/ro-phase)
    err_func = -0.5 * (erf(-1/np.sqrt(2)*(x-x0)/sigma - sigma/w) ) + 0.5
    return damp * err_func

def exp_decay(x, Amplitude, x0, sigma, w, oa, rg, ro, phase):
    fit1 = Amplitude/2*np.exp(-(x-x0)/w + ((sigma/w)**2)/2)*(1-erf(-1/np.sqrt(2)*(x-x0)/sigma - sigma/w))
    osc = damp_osci(x, x0, oa, rg, ro, phase, sigma, w)
    return fit1 + osc

def gaussian(x, Amplitude, x0, sigma):
    return Amplitude * np.exp(-np.power(x - x0, 2.) / (2 * np.power(sigma, 2.)))

gmodel = Model(exp_decay)

In [4]:
print(f'parameter names: {gmodel.param_names}')
print(f'independent variables: {gmodel.independent_vars}')

parameter names: ['Amplitude', 'x0', 'sigma', 'w', 'oa', 'rg', 'ro', 'phase']
independent variables: ['x']


In [5]:
params = gmodel.make_params(
    Amplitude=dict(value=0.06), 
    x0=dict(value=0.0), 
    sigma=dict(value=0.1), 
    w=dict(value=0.1), 
    oa=dict(value=0, vary=False),
    rg=dict(value=10, vary=False), 
    ro=dict(value=10, vary=False), 
    phase=dict(value=0, vary=False)
)
params

name,value,initial value,min,max,vary
Amplitude,0.06000000,0.06,-inf,inf,True
x0,0.00000000,0.0,-inf,inf,True
sigma,0.10000000,0.1,-inf,inf,True
w,0.10000000,0.1,-inf,inf,True
oa,0.00000000,0,-inf,inf,False
rg,10.0000000,10,-inf,inf,False
ro,10.0000000,10,-inf,inf,False
phase,0.00000000,0,-inf,inf,False


In [6]:
dataset_for_intensity = {}
dataset_for_peak_shift = {}

In [7]:
time_drifts = {}

In [ ]:
%matplotlib inline

In [8]:
plt.close('all')

In [ ]:
# ./scan.py delay th --delay -4 10 1 --delay 10 31 2 --th 27.415 25.815 80 --add-slow tth --tth 52.875 52.875 80

# Jungfrau ROI1 signal

In [9]:
run_n = 1
run_dir = f'/xfel/ffs/dat/ue_240531_FXS/raw_data/h5/type=measurement/run={run_n:03}'
scan_dirs = [scan_dir for scan_dir in sorted(os.listdir(run_dir)) if scan_dir.startswith('scan=')]
scan_numbers = [int(scan_dir.replace('scan=', '')) for scan_dir in scan_dirs]

for scan_dir in tqdm(scan_dirs):
    scan_n = int(scan_dir.replace('scan=', ''))
    fnames = [os.path.join(run_dir, scan_dir, fname) for fname in sorted(os.listdir(os.path.join(run_dir, scan_dir)))]

    labels = dict(
        position_label = '',
        pump_event_label = 'event_info.RATE_HX_30HZ',
        #signal_label = 'pd:es:pd1:ch2',
        signal_label = 'detector:eh1:jungfrau2:ROI1_stat.sum',
        #norm_label = 'pd:es:pd1:ch
        #norm_label = 'qbpm:oh:qbpm2:sum',
        norm_label = 'qbpm:eh1:qbpm1:sum',
        
        #orm2_label = 'detector:eh1:jungfrau2:ROI7_stat.sum',
      # newsignal_label = 'detector:eh1:jungfrau2:ROI1_stat.sum'
    )
    positions = []
    signals_on = []
    signals_off = []
    ps_on = []
    ps_off = []
 
    for fname in tqdm(fnames[:], leave=True):
        df = pd.read_hdf(fname)
        position_labels = [el for el in df.columns.values if el.endswith('_input')]
        for i, pl in enumerate(position_labels):
            labels[f'position_label_{i}'] = pl
        
        df_mini = df[labels.values()].dropna()
        position = [df_mini[labels[pl]].median() for pl in [el for el in labels.keys() if el.startswith('position_label')]]
        pump_event = df_mini[labels['pump_event_label']] == True
        
        #bkg_mean = (df_mini[labels['bkg_label']] - df_mini[labels['signal_label']]) / (df_mini[labels['roi2_area_label']] - df_mini[labels['roi1_area_label']])
        roi_sum = (
            df_mini[[
                labels['signal_label'],
            ]].values
            ).sum(axis=1)
        
        norm = df_mini[labels['norm_label']] #- bkg_mean*df_mini[labels['roi3_area_label']]

        norm_mask = np.logical_and(
            norm > norm.mean() - norm.std()*2,
            norm < norm.mean() + norm.std()*2,
        )
        norm_on = norm[norm_mask][pump_event[norm_mask]]
        norm_off = norm[norm_mask][pump_event[norm_mask] != True]
          
        signal_on = roi_sum[norm_mask][pump_event[norm_mask]] #- bkg_mean[pump_event] * df_mini[labels['roi1_area_label']][pump_event]
        signal_off = roi_sum[norm_mask][pump_event[norm_mask] != True] #- bkg_mean[pump_event != True] * df_mini[labels['roi1_area_label']][pump_event !=True]
               
        signal_ratio_on = signal_on / norm_on
        signal_ratio_off = signal_off / norm_off

        
        '''
        valid_on = np.logical_and(
            signal_ratio_on<signal_ratio_on.median() + signal_ratio_on.std()*.3, 
            signal_ratio_on>signal_ratio_on.median() - signal_ratio_on.std()*.3
            )
        valid_off = np.logical_and(
            signal_ratio_off<signal_ratio_off.median() + signal_ratio_off.std()*.3, 
            signal_ratio_off>signal_ratio_off.median() - signal_ratio_off.std()*.3
            )
        '''
        valid_on = np.ones(signal_ratio_on.shape, dtype = bool)
        valid_off = np.ones(signal_ratio_off.shape, dtype = bool)>signal_ratio_off.median() - signal_ratio_off.std()*.3
        
        p_on = np.polyfit(norm_off, signal_off, 1)
        p_off = np.polyfit(norm_on, signal_on, 1)
        
        '''
        signal_norm_on = 1/p_on[0]
        signal_norm_off = 1/p_off[0]
        '''
        '''
        signal_norm_on = np.average(
            signal_on / norm_on,
            weights = norm_on
            )
        signal_norm_off = np.average(
            signal_off / norm_off,
            weights = norm_off
            )
        '''
        signal_norm_on = np.average(
            signal_on[valid_on] /norm_on[valid_on],
            weights = norm_on[valid_on]
            )
        signal_norm_off = np.average(
            signal_off[valid_off] / norm_off[valid_off],
            weights = norm_off[valid_off]
            )
        
        
        
        '''
        p = np.polyfit(norm, roi_sum, 2)
        params['a_on'].value = p[0]
        params['a_off'].value = p[0]
        params['b'].value = p[1]
        
        out = minimize(residual, params, args=(norm_on, signal_on, norm_off, signal_off))

        signal_norm_on = out.params['a_on'].value
        signal_norm_off = out.params['a_off'].value
        '''
        
        positions.append(position)
        signals_on.append(signal_norm_on)
        signals_off.append(signal_norm_off)
        ps_on.append(p_on)
        ps_off.append(p_off)

        
    positions = np.array(positions)
    signals_on = np.array(signals_on)
    signals_off = np.array(signals_off)
    signals_diff = (signals_on - signals_off)#/signals_off
    
    ps_on = np.array(ps_on)
    ps_off = np.array(ps_off)
    
    dataset_for_intensity[(run_n, scan_n)] = \
        {
            "labels": labels,
            "positions": positions,
            "signals_on": signals_on,
            "signals_off": signals_off,
            "signals_diff": signals_diff,
            "ps_on": ps_on,
            "ps_off": ps_off, 
        }

plt.close('all')

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '/xfel/ffs/dat/ue_240531_FXS/raw_data/h5/type=measurement/run=020'

In [ ]:
position_labels

In [ ]:
x_axis_label = 'delay_input'
y_axis_label = 'th_input'

In [ ]:
x_axis_idx

In [ ]:
x_axis_idx = position_labels.index(x_axis_label)
y_axis_idx = position_labels.index(y_axis_label)

In [ ]:
plt.figure()
plt.imshow(
    signals_on.reshape(np.unique(positions[:, y_axis_idx]).size, np.unique(positions[:, x_axis_idx]).size),
    interpolation='gaussian', aspect='auto',
    extent=[
        np.unique(positions[:, x_axis_idx])[0], np.unique(positions[:, x_axis_idx])[-1],
        np.unique(positions[:, y_axis_idx])[-1], np.unique(positions[:, y_axis_idx])[0]
    ]
)
plt.xlabel(x_axis_label)
plt.ylabel(y_axis_label)
plt.show()

In [ ]:
plt.figure()
plt.imshow(
    signals_off.reshape(np.unique(positions[:, y_axis_idx]).size, np.unique(positions[:, x_axis_idx]).size),
    interpolation='gaussian', aspect='auto',
    extent=[
        np.unique(positions[:, x_axis_idx])[0], np.unique(positions[:, x_axis_idx])[-1],
        np.unique(positions[:, y_axis_idx])[-1], np.unique(positions[:, y_axis_idx])[0]
    ]
)
plt.xlabel(x_axis_label)
plt.ylabel(y_axis_label)
plt.show()

In [ ]:
plt.figure()
plt.imshow(
    signals_diff.reshape(np.unique(positions[:, y_axis_idx]).size, np.unique(positions[:, x_axis_idx]).size),
    interpolation='gaussian', aspect='auto',
    extent=[
        np.unique(positions[:, x_axis_idx])[0], np.unique(positions[:, x_axis_idx])[-1],
        np.unique(positions[:, y_axis_idx])[-1], np.unique(positions[:, y_axis_idx])[0]
    ]
)
plt.xlabel(x_axis_label)
plt.ylabel(y_axis_label)
plt.colorbar()
plt.show()